In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import joblib  # for saving the model

# Load dataset
A = pd.read_csv("dream_reality_overlap_12cols.csv")

# Drop ID columns
data = A.drop(columns=['Record_ID', 'Participant_ID'])

# Handle NaN in Lucidity_Type by filling with 'None' string before encoding
data['Lucidity_Type'] = data['Lucidity_Type'].fillna('None')

# Encode categorical columns
label_encoders = {}
for col in data.select_dtypes(include='object').columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Separate features (X) and target (y)
x = data.drop(columns=['Dream_Reality_Overlap_Score_0_100'])
y = data['Dream_Reality_Overlap_Score_0_100']

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Train Random Forest model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(x_train, y_train)

# Predictions
y_pred = model.predict(x_test)

# Evaluate model
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"✅ Model Performance:")
print(f"R² Score: {r2:.4f}")
print(f"Mean Absolute Error: {mae:.4f}")
print(f"Root Mean Squared Error: {rmse:.4f}")

# Save the model and label encoders
joblib.dump(model, 'dream_overlap_model.pkl')
joblib.dump(label_encoders, 'label_encoders.pkl')

print("\n💾 Model saved successfully as 'dream_overlap_model.pkl'")

FileNotFoundError: [Errno 2] No such file or directory: 'dream_reality_overlap_12cols.csv'

In [ ]:
A

,Record_ID,Participant_ID,Age,Gender,Sleep_Duration_hours,REM_Duration_minutes,Dream_Vividness_1_10,Lucidity_Type,Dream_Theme,Primary_Emotion,Dream_Reality_Overlap_Score_0_100,Sleep_Quality_1_10
0,R0000001,P023655,73,Male,7.20,63.4,10,NaN,Chase,Confused,73.35,8.22
1,R0000002,P015796,20,Female,5.19,58.9,8,NaN,Everyday,Happy,59.91,6.95
2,R0000003,P000861,45,Female,7.05,60.2,7,NaN,Exam/Work,Happy,53.94,7.04
3,R0000004,P005391,70,Male,5.76,124.5,4,NaN,Loved ones,Confused,36.05,6.68
4,R0000005,P021576,47,Female,6.33,92.1,7,NaN,Everyday,Happy,55.84,6.26
...,...,...,...,...,...,...,...,...,...,...,...,...
119995,R0119996,P017750,30,Male,6.46,105.3,4,NaN,Flying,Neutral,34.38,6.62
119996,R0119997,P019392,21,Male,9.27,119.5,8,NaN,Everyday,Anxious,63.44,5.54
119997,R0119998,P023861,19,Female,8.10,61.0,4,NaN,Loved ones,Neutral,28.79,7.82
119998,R0119999,P001052,38,Female,6.10,26.1,7,Trained,Nightmare,Happy,60.54,4.95


In [ ]:
%pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 78.5 MB/s eta 0:00:00


In [ ]:
%%writefile dream_reality_overlap.py
import streamlit as st
import pickle
import numpy as np

Writing dream_reality_overlap.py


In [ ]:
import streamlit as st
import pandas as pd
import joblib

# Load model and encoders
model = joblib.load('dream_overlap_model.pkl')
label_encoders = joblib.load('label_encoders.pkl')

# Streamlit App
st.title("🌙 Dream–Reality Overlap Predictor")
st.markdown("Predict how much your dream overlaps with reality based on your sleep and dream characteristics.")

# User inputs
age = st.number_input("Age", min_value=10, max_value=100, value=25)
gender = st.selectbox("Gender", ["Male", "Female", "Other"])
sleep_duration = st.number_input("Sleep Duration (hours)", min_value=0.0, max_value=12.0, value=7.0)
rem_duration = st.number_input("REM Duration (minutes)", min_value=0.0, max_value=200.0, value=90.0)
vividness = st.slider("Dream Vividness (1–10)", 1, 10, 7)
lucidity = st.selectbox("Lucidity Type", ["None", "Partial", "Full"])
theme = st.selectbox("Dream Theme", ["Everyday", "Adventure", "Loved ones", "Chase", "Exam/Work", "Other"])
emotion = st.selectbox("Primary Emotion", ["Happy", "Fear", "Confused", "Excited", "Sad", "Other"])
sleep_quality = st.slider("Sleep Quality (1–10)", 1.0, 10.0, 7.0)

# Prepare input data
input_data = pd.DataFrame([{
    'Age': age,
    'Gender': gender,
    'Sleep_Duration_hours': sleep_duration,
    'REM_Duration_minutes': rem_duration,
    'Dream_Vividness_1_10': vividness,
    'Lucidity_Type': lucidity,
    'Dream_Theme': theme,
    'Primary_Emotion': emotion,
    'Sleep_Quality_1_10': sleep_quality
}])

# Encode categorical columns using saved encoders
for col, le in label_encoders.items():
    if col in input_data.columns:
        input_data[col] = le.transform(input_data[col])

# Predict
if st.button("🔮 Predict Dream–Reality Overlap"):
    prediction = model.predict(input_data)[0]
    st.success(f"**Predicted Dream–Reality Overlap Score:** {prediction:.2f} / 100")

    # Insight message
    if prediction > 75:
        st.info("✨ High overlap – you might experience strong dream-reality blending or vivid dream recall.")
    elif prediction > 50:
        st.info("🌗 Moderate overlap – your dreams may influence your waking thoughts occasionally.")
    else:
        st.info("💤 Low overlap – your dreams and reality are mostly distinct.")

st.markdown("---")
st.caption("Created with ❤️ using Streamlit and Machine Learning.")


2025-11-12 15:41:39.945 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-12 15:41:39.946 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-12 15:41:39.947 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-12 15:41:39.949 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-12 15:41:39.951 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-12 15:41:39.953 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-12 15:41:39.955 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-12 15:41:39.956 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()